In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
# login information to be passed into HTML requests. DO NOT DIRECTLY PUSH LOGIN INFORMATION TO YOUR GITHUB REPOSITORY!
username = ""
password = ""

payload = {
    "ctl00$cph1$ls1$txtEmail" : username,
    "ctl00$cph1$ls1$txtPassword" : password,
    "ctl00$cph1$ls1$btnLogin": "Login"
}

In [4]:
url = "http://www.eoddata.com/symbols.aspx"

# use Session() to make multiple requests
with requests.Session() as s:
    page = s.get(url)
    soup = BeautifulSoup(page.content)
    # additional form data passed for ASP.NET based website
    payload["__VIEWSTATE"] = soup.select_one("#__VIEWSTATE")["value"]
    payload["__VIEWSTATEGENERATOR"] = soup.select_one("#__VIEWSTATEGENERATOR")["value"]
    payload["__EVENTVALIDATION"] = soup.select_one("#__EVENTVALIDATION")["value"]
    
    res = s.post(url, data=payload)
    
    validation = BeautifulSoup(res.content)
    logout = validation.select_one("#ctl00_cph1_ls1_lnkLogOut")
    
    if logout is None:
        print("Log in failed")
        
    elif logout.text.lower() == 'log out':
        # get .txt formed data
        nyse_txt = s.get("http://www.eoddata.com/Data/symbollist.aspx?e=NYSE")
        nasdaq_txt = s.get("http://www.eoddata.com/Data/symbollist.aspx?e=NASDAQ")
        amex_txt = s.get("http://www.eoddata.com/Data/symbollist.aspx?e=AMEX")

In [9]:
def convert_to_dataframe(text_data, exchange_name):
    row_splitted = text_data.split('\r\n')
    exchange = pd.DataFrame([x.split('\t') for x in row_splitted], columns=['Symbol', 'Description'])
    # remove duplicated header and blank row at the end
    exchange = exchange.drop([exchange.index[0], exchange.index[-1]])
    # add which exchange
    name = pd.Series([exchange_name], name='Exchange')
    exchange = exchange.merge(name, how='cross')
    
    return(exchange)

In [10]:
nyse = convert_to_dataframe(nyse_txt.text, 'NYSE')
nasdaq = convert_to_dataframe(nasdaq_txt.text, 'NASDAQ')
amex = convert_to_dataframe(amex_txt.text, 'AMEX')

In [12]:
nyse

pandas.core.frame.DataFrame

In [122]:
nasdaq

,Symbol,Description,exchange
0,AACG,Ata Creativity Global ADR,NASDAQ
1,AACQ,Artius Acquisition Inc Cl A,NASDAQ
2,AACQU,Artius Acquisition Inc Unit,NASDAQ
3,AACQW,Artius Acquisition Inc WT,NASDAQ
4,AADR,Advisorshares Dorsey Wright ETF,NASDAQ
...,...,...,...
4826,ZWRKU,Z-Work Acquisition Corp Units,NASDAQ
4827,ZWRKW,Z-Work Acquisition Corp WT,NASDAQ
4828,ZY,Zymergen Inc,NASDAQ
4829,ZYNE,Zynerba Pharma CS,NASDAQ


In [123]:
amex

,Symbol,Description,exchange
0,AAA,First Priority Clo Bond ETF,AMEX
1,AAAU,GS Physical Gold ETF,AMEX
2,AADR,Advisorshares Dorsey Wright ETF,AMEX
3,AAMC,Altisource Asset,AMEX
4,AAU,Almaden Minerals,AMEX
...,...,...,...
2450,ZDGE,Zedge Inc Cl B,AMEX
2451,ZIG,The Acquirers Fund ETF,AMEX
2452,ZOM,Zomedica Pharmaceuticals Corp,AMEX
2453,ZROZ,25+ Year Zero Cpn U.S. Trsy Idx ETF Pimco,AMEX


In [15]:
# make as one dataframe
exchange_list = [nasdaq, amex]
stock_list = nyse.append(exchange_list, ignore_index=True)

In [16]:
# check duplicated stocks across three different exchange
stock_list[stock_list.duplicated()]

,Symbol,Description,Exchange


In [17]:
stock_list = stock_list.drop_duplicates(subset=['Symbol'], ignore_index=True)

In [18]:
stock_list.to_csv("stock_list_210705.csv", index=False)

In [19]:
# check changes in stock list
old = pd.read_csv("stock_list_210701.csv")

In [20]:
print(f"yesterday : {len(old)}, today : {len(stock_list)}")

yesterday : 10929, today : 10967


In [21]:
changes = pd.merge(old, stock_list, how='outer', indicator=True)
changes[changes['_merge']!='both']

,Symbol,Description,Exchange,_merge
188,AMH-D,American Homes 4 Rent [Amh/Pd],NYSE,left_only
737,CLGX,Corelogic,NYSE,left_only
880,CTB,Cooper Tire & Rubber Company,NYSE,left_only
1886,JWS,Jaws Acquisition Corp Cl A.,NYSE,left_only
1889,JWS.U,Jaws Acquisition Corp [Jws.U],NYSE,left_only
...,...,...,...,...
10980,QVMM,Invesco S&P Midcap 400 Qvm Multi-Factor ETF,AMEX,right_only
10981,QVMS,Invesco S&P Smallcap 600 Qvm Multi-Factor ETF,AMEX,right_only
10982,XBJL,Innovator U.S. Equity Accelerated 9 Buffer ETF J,AMEX,right_only
10983,XDJL,Innovator U.S. Equity Accelerated ETF July,AMEX,right_only


In [22]:
# search stock
stock_list[stock_list['Symbol']=='CTB']

,Symbol,Description,Exchange
